In [ ]:
import pandas as pd
import numpy as np
import importlib
import gc
from sklearn.preprocessing import StandardScaler, MinMaxScaler  # Cambiamos el import
from prophet import Prophet
import sys
sys.path.append('./scripts')  
import feature_engineering
importlib.reload(feature_engineering)

In [95]:
df = pd.read_csv("./datasets/periodo_x_producto_con_target_transformado.csv", sep=',', encoding='utf-8')
df

,product_id,periodo,nacimiento_producto,muerte_producto,mes_n,total_meses,producto_nuevo,ciclo_de_vida_inicial,sku_size,stock_final,...,brand_nan,tn_zscore,stock_final_zscore,cust_request_qty_zscore,cust_request_tn_zscore,target,tn_log,stock_final_log,cust_request_qty_log,cust_request_tn_log
0,20001,201701,201701,201912,1,36,0,0,3000.0,NaN,...,0.0,-1.492929,NaN,0.402760,-1.616977,-0.450817,6.841372,NaN,6.173786,6.844525
1,20001,201702,201701,201912,2,36,0,0,3000.0,NaN,...,0.0,-1.941219,NaN,-0.091059,-1.940746,-1.299333,6.683381,NaN,6.070738,6.727099
2,20001,201703,201701,201912,3,36,0,0,3000.0,NaN,...,0.0,-0.284694,NaN,0.717964,-0.393505,0.272083,7.173466,NaN,6.234411,7.194247
3,20001,201704,201701,201912,4,36,0,0,3000.0,NaN,...,0.0,-1.049775,NaN,-1.698597,-1.009266,0.337028,6.976312,NaN,5.634790,7.033457
4,20001,201705,201701,201912,5,36,0,0,3000.0,NaN,...,0.0,0.367121,NaN,2.735266,0.291176,-1.442163,7.315352,NaN,6.553933,7.347100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31357,21281,201704,201702,201708,3,7,1,1,NaN,NaN,...,1.0,-0.670410,NaN,NaN,NaN,1.850294,0.000000,NaN,NaN,NaN
31358,21281,201705,201702,201708,4,7,1,0,NaN,NaN,...,1.0,-0.670410,NaN,NaN,NaN,-0.707610,0.000000,NaN,NaN,NaN
31359,21281,201706,201702,201708,5,7,1,0,NaN,NaN,...,1.0,2.237248,NaN,1.414214,1.397345,0.272537,0.087406,NaN,2.197225,0.100198
31360,21281,201707,201702,201708,6,7,1,0,NaN,NaN,...,1.0,-0.670410,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN


In [ ]:
# Revisar si hay infinitos en la columna target
print("Valores infinitos en 'tn_zscore':", np.isinf(df['tn_zscore']).sum())

# Revisar también NaN
print("Valores NaN en 'tn_zscore':", df['tn_zscore'].isna().sum())

df['tn_zscore'] = df['tn_zscore'].replace([np.inf, -np.inf], np.nan)
# df['tn_zscore'] = df['tn_zscore'].fillna(df['tn_zscore'].median())  # Usar la mediana
df['tn_zscore'] = df['tn_zscore'].fillna(0)  # Usar cero #### EXPERIMENTAR CON ESTO

df

In [77]:
df[df['product_id'] == 20001]

,product_id,periodo,autocorrelacion_lag_1,autocorrelacion_lag_2,autocorrelacion_lag_3,autocorrelacion_lag_4,autocorrelacion_lag_5,autocorrelacion_lag_6,autocorrelacion_lag_7,autocorrelacion_lag_8,...,autocorrelacion_lag_27,autocorrelacion_lag_28,autocorrelacion_lag_29,autocorrelacion_lag_30,autocorrelacion_lag_31,autocorrelacion_lag_32,autocorrelacion_lag_33,autocorrelacion_lag_34,autocorrelacion_lag_35,autocorrelacion_lag_36
0,20001,201912,0.213196,0.185552,-0.050078,-0.022983,0.278439,0.119253,0.1757,-0.151486,...,-0.482787,0.673611,-0.113422,0.117276,0.063798,-0.803158,0.426375,-1.0,NaN,NaN


In [ ]:
##### Aca puedo agregar todas las columnas que quiero usar como features 

#### cuidado con el data leakage, no usar el target en el entrenamiento
df = feature_engineering.get_lags(df, "tn_zscore", 201912)
df = feature_engineering.get_delta_lags(df, "tn_zscore", 201912)
df = feature_engineering.get_rolling_means(df, "tn_zscore", 201912)
df = feature_engineering.get_rolling_stds(df, "tn_zscore", 201912)
df = feature_engineering.get_rolling_mins(df, "tn_zscore", 201912)
df = feature_engineering.get_rolling_maxs(df, "tn_zscore", 201912)
df = feature_engineering.get_autocorrelaciones(df, "tn_zscore", 201912)

In [35]:
df = feature_engineering.generar_ids(df)
df = feature_engineering.get_componentesTemporales(df)

In [36]:
##### TARDA 66 MINUTOS EN EJECUTARSE

estoy_en_kaggle = False

if estoy_en_kaggle:
    df = feature_engineering.get_prophet_features(df, target_col='tn_zscore') ###  no sus target porque estaria haciendo data leakage
else:
    df = feature_engineering.get_prophet_features_sin_data_leakage(df, target_col='tn_zscore', train_mask = (df['periodo'] <= 201909))

16:32:09 - cmdstanpy - INFO - Chain [1] start processing
16:32:09 - cmdstanpy - INFO - Chain [1] done processing
16:32:10 - cmdstanpy - INFO - Chain [1] start processing
16:32:10 - cmdstanpy - INFO - Chain [1] done processing
c:\Users\Usuario\Documents\Universidad\austral\2025\Lab3\Lab3-MCD\notebooks\entregable\./scripts\feature_engineering.py:395: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_features[feature_cols] = scaler.fit_transform(train_features[feature_cols])
c:\Users\Usuario\Documents\Universidad\austral\2025\Lab3\Lab3-MCD\notebooks\entregable\./scripts\feature_engineering.py:397: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

In [37]:
df[df['product_id'] == 20060].filter(regex='^(?!cat)')

,product_id,periodo,nacimiento_producto,muerte_producto,mes_n,total_meses,producto_nuevo,ciclo_de_vida_inicial,sku_size,stock_final,...,dia_del_year,dia_del_year_sin,dia_del_year_cos,ds,trend_add,yearly_add,additive_terms,trend_mult,yearly_mult,multiplicative_terms
2079,20060,201701,201701,201905,1,29,0,0,800.0,NaN,...,1,0.017213,0.999852,2017-01-01,1.670359,-1.780342,-1.780342,1.670359,-1.706775,-1.706775
2080,20060,201702,201701,201905,2,29,0,0,800.0,NaN,...,32,0.523416,0.852078,2017-02-01,1.548531,-1.558282,-1.558282,1.548531,-1.757946,-1.757946
2081,20060,201703,201701,201905,3,29,0,0,800.0,NaN,...,60,0.858764,0.512371,2017-03-01,1.438492,0.958228,0.958228,1.438492,0.050403,0.050403
2082,20060,201704,201701,201905,4,29,0,0,800.0,NaN,...,91,0.999991,0.004304,2017-04-01,1.316664,0.259218,0.259218,1.316664,-0.049612,-0.049612
2083,20060,201705,201701,201905,5,29,0,0,800.0,NaN,...,121,0.871706,-0.490029,2017-05-01,1.198766,0.544768,0.544768,1.198766,-0.649513,-0.649513
2084,20060,201706,201701,201905,6,29,0,0,800.0,NaN,...,152,0.501242,-0.865307,2017-06-01,1.076938,0.237052,0.237052,1.076938,-0.036208,-0.036208
2085,20060,201707,201701,201905,7,29,0,0,800.0,NaN,...,182,0.008607,-0.999963,2017-07-01,0.959040,1.793749,1.793749,0.959040,1.470718,1.470718
2086,20060,201708,201701,201905,8,29,0,0,800.0,NaN,...,213,-0.501242,-0.865307,2017-08-01,0.837212,-0.171366,-0.171366,0.837212,-0.205690,-0.205690
2087,20060,201709,201701,201905,9,29,0,0,800.0,NaN,...,244,-0.871706,-0.490029,2017-09-01,0.715384,1.761853,1.761853,0.715384,2.007232,2.007232
2088,20060,201710,201701,201905,10,29,0,0,800.0,NaN,...,274,-0.999991,0.004304,2017-10-01,0.597486,-0.223172,-0.223172,0.597486,-0.101224,-0.101224


In [41]:
df[df['product_id'] == 20001].filter(regex='_rolling_mean_')

,tn_zscore_rolling_mean_1,tn_zscore_rolling_mean_2,tn_zscore_rolling_mean_3,tn_zscore_rolling_mean_4,tn_zscore_rolling_mean_5,tn_zscore_rolling_mean_6,tn_zscore_rolling_mean_7,tn_zscore_rolling_mean_8,tn_zscore_rolling_mean_9,tn_zscore_rolling_mean_10,...,tn_zscore_rolling_mean_27,tn_zscore_rolling_mean_28,tn_zscore_rolling_mean_29,tn_zscore_rolling_mean_30,tn_zscore_rolling_mean_31,tn_zscore_rolling_mean_32,tn_zscore_rolling_mean_33,tn_zscore_rolling_mean_34,tn_zscore_rolling_mean_35,tn_zscore_rolling_mean_36
0,-1.492929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-1.941219,-1.717074,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.284694,-1.112957,-1.239614,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-1.049775,-0.667235,-1.091896,-1.192154,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.367121,-0.341327,-0.322449,-0.727142,-0.880299,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.425680,0.396400,-0.085658,-0.135417,-0.496578,-0.662636,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,-1.178560,-0.376440,-0.128586,-0.358884,-0.344046,-0.610241,-0.736339,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,-0.402583,-0.790571,-0.385154,-0.197085,-0.367623,-0.353802,-0.580576,-0.694620,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,-0.240151,-0.321367,-0.607098,-0.348904,-0.205699,-0.346378,-0.337566,-0.538023,-0.644123,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.162422,-0.038865,-0.160104,-0.414718,-0.246638,-0.144345,-0.273692,-0.275068,-0.460196,-0.563469,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df = feature_engineering.get_anomaliasPoliticas(df)
df = feature_engineering.get_IPC(df)
df = feature_engineering.get_dolar(df)
df = feature_engineering.get_mes_receso_escolar(df)
df = feature_engineering.mes_con_feriado(df)

In [ ]:
df.to_csv("./datasets/periodo_x_producto_con_target_transformado_con_feature_engineering.csv", index=False, sep=',', encoding='utf-8')